## Converting PyTorch model to ONNX model:

Saving ONNX model to `onnx_bert_large/bert-large-nli-mean-tokens-onnx.onnx`

In [2]:
#!rm -rf onnx_bert_large/
from transformers.convert_graph_to_onnx import convert
convert(framework="pt", model="./", output="onnx_bert_large/bert-large-nli-mean-tokens-onnx.onnx", opset=11)

f:\GitHub\onnx_export\venv\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(
f:\GitHub\onnx_export\venv\Lib\site-packages\transformers\convert_graph_to_onnx.py:361: FutureWarning: The `transformers.convert_graph_to_onnx` package is deprecated and will be removed in version 5 of Transformers
  warnings.warn(


ONNX opset version set to: 11
Loading pipeline (model: ./, tokenizer: ./)


AttributeError: 'str' object has no attribute 'parent'

## Creating an ONNX Inference Session

In [3]:
from os import environ
from psutil import cpu_count

# Constants from the performance optimization available in onnxruntime
# It needs to be done before importing onnxruntime
environ["OMP_NUM_THREADS"] = str(cpu_count(logical=True))
environ["OMP_WAIT_POLICY"] = 'ACTIVE'

from onnxruntime import InferenceSession, SessionOptions, get_all_providers

In [4]:
sess = InferenceSession("onnx/model.onnx", providers=["CPUExecutionProvider"])

## Initializing mean_pooling function to convert model sequence outputs to pooled outputs

In [5]:
import torch

In [6]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    token_embeddings = model_output #First element of model_output contains all token embeddings
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

## Initializing BERT Tokenizer

In [7]:
from transformers import BertTokenizerFast
# Using bert-base-uncased because Sentence Transformers uses the same
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

f:\GitHub\onnx_export\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Getting sentence embeddings using the inference session

In [8]:
query = "My name is Bert"

Tokenizing and getting into format required for ONNX Model

In [19]:
model_inputs = tokenizer(query, return_tensors="pt")
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}

Running the inference session with `inputs_onnx`

In [20]:
sequence = sess.run(None, inputs_onnx)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid input name: token_type_ids

Converting sequence embeddings to pooled embeddings using `mean_pooling` function

In [11]:
sentence_embeddings = mean_pooling(sequence, inputs_onnx['attention_mask'])

NameError: name 'sequence' is not defined

### Sentence embeddings:

In [12]:
sentence_embeddings[0]

NameError: name 'sentence_embeddings' is not defined

In [13]:
sentence_embeddings

NameError: name 'sentence_embeddings' is not defined

In [14]:
np.shape(sentence_embeddings[0])

NameError: name 'np' is not defined

In [35]:
np.shape(sentence_embeddings[1])

torch.Size([1, 6, 1024])

In [36]:
np.shape(sentence_embeddings[2])

torch.Size([1, 1024])